In [98]:
#python example to infer document vectors from trained doc2vec model
import gensim.models as g
import codecs
from scipy import spatial
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

In [3]:
def create_dataframe(contest,dvec_model,g_truth):

    df = OrderedDict()
    file_class =[]
    csv = pd.read_csv('../Data/'+str(contest)+'/standings_statistics.csv',sep='\t',skiprows=1,header=None,names=['ParticipantID','Rank','Country','ProblemA_id','ProblemA_memory','ProblemA_time','ProblemA_language','ProblemB_id','ProblemB_memory','ProblemB_time','ProblemB_language','ProblemC_id','ProblemC_memory','ProblemC_time','ProblemC_language','ProblemD_id','ProblemD_memory','ProblemD_time','ProblemD_language','ProblemE_id','ProblemE_memory','ProblemE_time','ProblemE_language','ProblemF_id','ProblemF_memory','ProblemF_time','ProblemF_language','ProblemG_id','ProblemG_memory','ProblemG_time','ProblemG_language'])
    for i in os.listdir('../Data/'+str(contest)+'/all_xml_simplified/'):
        f_xml_string = open('../Data/'+str(contest)+'/all_xml_simplified/'+str(i),'r').read()
        f_xml_list = f_xml_string.strip().split()
        df[i] = dvec_model.infer_vector(f_xml_list)
        
        file_id = str(i).strip('.xml').split("_")[1]
        file_id = str(file_id)
        
        if g_truth=='problem':
            if file_id in csv['ProblemA_id'].to_list():
                file_class.append(0)
            elif file_id in csv['ProblemB_id'].to_list():
                file_class.append(1)
            elif file_id in csv['ProblemC_id'].to_list():
                file_class.append(2)
            elif file_id in csv['ProblemD_id'].to_list():
                file_class.append(3)
            elif file_id in csv['ProblemE_id'].to_list():
                file_class.append(4)
            elif file_id in csv['ProblemF_id'].to_list():
                file_class.append(5)
            elif file_id in csv['ProblemG_id'].to_list():
                file_class.append(6)
    return pd.DataFrame(df),file_class

def cluster_vecs(n_clusters,data):
    kmeans_model = KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_clusters,n_jobs=-1)
    mdl = kmeans_model.fit(data)
    return mdl

def intersection(a,b):
    return len(list(set(a) & set(b)))

In [4]:
#parameters
model="model.bin"
#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000
#load model
m = g.Doc2Vec.load(model)


### Making vector representation of codes 

In [114]:
#ground truth - problem
x,labels = create_dataframe("Contest1244",m,'problem')

In [120]:
data = np.array(x)
data = data.T


In [129]:
data

array([[ 3.79978120e-02, -1.20230302e-01,  4.37393151e-02, ...,
        -1.64432257e-01, -9.93256345e-02,  3.01404208e-01],
       [-1.35547653e-01,  1.09405540e-01,  2.04108998e-01, ...,
         7.38251209e-02,  1.10557117e-01,  7.35832155e-02],
       [-2.76787311e-01, -1.67579651e-01,  2.26770625e-01, ...,
        -2.80112654e-01,  1.96913943e-01,  2.49382183e-01],
       ...,
       [-3.05308029e-02,  2.77704035e-04,  8.06697384e-02, ...,
        -2.13191628e-01, -1.02161624e-01,  2.32980177e-01],
       [-9.69470814e-02,  1.51156187e-01,  9.65372548e-02, ...,
        -3.26345023e-03, -8.60722587e-02, -6.02240041e-02],
       [-5.79341352e-02, -2.53210217e-01,  1.90019980e-01, ...,
        -2.16098294e-01, -1.19963355e-01,  1.24417087e-02]], dtype=float32)

## K-means on Doc2Vecs of codes

In [122]:
km_model = cluster_vecs(7,data)
pred_labels = km_model.labels_

In [123]:
pred_A = [i for i, x in enumerate(pred_labels) if x == 0]
pred_B = [i for i, x in enumerate(pred_labels) if x == 1]
pred_C = [i for i, x in enumerate(pred_labels) if x == 2]
pred_D = [i for i, x in enumerate(pred_labels) if x == 3]
pred_E = [i for i, x in enumerate(pred_labels) if x == 4]
pred_F = [i for i, x in enumerate(pred_labels) if x == 5]
pred_G = [i for i, x in enumerate(pred_labels) if x == 6]

true_A = [i for i, x in enumerate(labels) if x == 0]
true_B = [i for i, x in enumerate(labels) if x == 1]
true_C = [i for i, x in enumerate(labels) if x == 2]
true_D = [i for i, x in enumerate(labels) if x == 3]
true_E = [i for i, x in enumerate(labels) if x == 4]
true_F = [i for i, x in enumerate(labels) if x == 5]
true_G = [i for i, x in enumerate(labels) if x == 6]





In [124]:
conf={}
for i in range(len(pred_labels)):
    if pred_labels[i] not in conf.keys():
        conf[pred_labels[i]]=[0,0,0,0,0,0,0]
        
    else:
        conf[pred_labels[i]][labels[i]]+=1

In [125]:
for key,value in conf.items():
    conf[key][0] = conf[key][0]/len(true_A)
    conf[key][1] = conf[key][1]/len(true_B)
    conf[key][2] = conf[key][2]/len(true_C)
    conf[key][3] = conf[key][3]/len(true_D)
    conf[key][4] = conf[key][4]/len(true_E)
    conf[key][5] = conf[key][5]/len(true_F)
    conf[key][6] = conf[key][6]/len(true_G)
    
    

In [126]:
df_confusion = pd.DataFrame(conf)

In [127]:
df_confusion = df_confusion.reindex(sorted(df_confusion.columns), axis=1)

In [128]:
df_confusion

,0,1,2,3,4,5,6
0,0.001049,0.088143,0.116474,0.020986,0.002099,0.469045,0.301154
1,0.003148,0.121721,0.137461,0.338930,0.025184,0.117524,0.254984
2,0.006006,0.093093,0.072072,0.201201,0.009009,0.330330,0.282282
3,0.690844,0.057075,0.023781,0.011891,0.210464,0.000000,0.004756
4,0.059642,0.083499,0.085487,0.504970,0.121272,0.033797,0.107356
5,0.189873,0.101266,0.063291,0.468354,0.151899,0.000000,0.025316
6,0.145455,0.127273,0.054545,0.418182,0.127273,0.018182,0.109091


## Logistic Regression approach (Multi-class classification)

In [132]:
labels = np.array(labels)
labels.shape

(3717,)

In [189]:
data_train = data[:round(0.6*(data.shape[0]))]
labels_train = labels[:round(0.6*(labels.shape[0]))]

In [190]:
data_test = data[round(0.6*(data.shape[0])):]
labels_test = labels[round(0.6*(labels.shape[0])):]

In [191]:
print(data_train.shape,(labels_train.shape),"+",data_test.shape,labels_test.shape)

(2230, 100) (2230,) + (1487, 100) (1487,)


In [192]:
class_weight=compute_class_weight('balanced', np.arange(0,7), labels)

In [193]:
cw_dict = {}
for i in range(len(class_weight)):
    cw_dict[i] = class_weight[i]

In [194]:
cw_dict

{0: 0.5571878279118573,
 1: 0.5571878279118573,
 2: 1.5945945945945945,
 3: 0.6313912009512486,
 4: 1.0556660039761432,
 5: 6.7215189873417724,
 6: 9.654545454545454}

In [195]:
log_reg = LogisticRegression(multi_class='multinomial',solver='lbfgs',n_jobs=-1,class_weight=cw_dict)

In [196]:
log_reg.fit(data_train,labels_train)

LogisticRegression(C=1.0,
                   class_weight={0: 0.5571878279118573, 1: 0.5571878279118573,
                                 2: 1.5945945945945945, 3: 0.6313912009512486,
                                 4: 1.0556660039761432, 5: 6.7215189873417724,
                                 6: 9.654545454545454},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='multinomial',
                   n_jobs=-1, penalty='l2', random_state=None, solver='lbfgs',
                   tol=0.0001, verbose=0, warm_start=False)

In [198]:
accuracy_score(labels_test,log_reg.predict(data_test))

0.8137188971082717